In [2]:
import nltk 
from nltk.tokenize import word_tokenize 
import numpy as np
import random
import re
import string
import sys
import pandas as pd
from collections import Counter
punc = string.punctuation


# Get Words to make Vocabulary

In [3]:
import pandas as pd
data = pd.read_csv(r'C:\Users\Salma\Downloads\sentence_error.csv')
with open(r'C:\Users\Salma\Downloads\Arabic-Wordlist-1.6\Arabic-Wordlist-1.6\arabic-wordlist-1.6.txt','r',encoding="utf8") as word_list:
    arabic_Correct_words=word_list.readlines()

In [4]:
wordsList=list()
for word in arabic_Correct_words:
    wordsList.append(word[:-1])

In [5]:
corpus = data['sentence']

In [6]:
corpus

0       أكثر من التخصص الحمد لله الذي وفقني لدراسة شر...
1       رحلة إلى أبها قمنا برحلة إلى أبها ، كنا تسعة ...
2       دراستي كلية أصول الدين عندي رغبة شديدة في درا...
3       أحلامي في البنك الإسلامي أنني أفضل كلية الاقت...
4       الرحلة إلى القرية لزيارة ذوي القربى اعتدت الذ...
                             ...                        
305                                          izona225_c1
306                                          izona232_c3
307                                          izona241_c1
308                                          izona239_c1
309                                          izona232_c1
Name: sentence, Length: 310, dtype: object

In [7]:
lines = [x for x in data['sentence']]
print(lines)

[' أكثر من التخصص الحمد لله الذي وفقني لدراسة شرعية في جامعة الإمام محمد بن سعود الإسلامية فأنا الآن أدرس في المستوى الثاني من قسم اللغة العربية والعلوم الإسلامية وأريد الالتحاق بكلية الشريعة إن شاء الله . واختيار الناس لهذه الكلية قد تختلف من شخص إلى آخر . قد يختار البعض من أجل وظيفة يحصل عليها بعد انتهاء الكلية والبعض الآخر يختار هذه الكلية لأن والديه يريدان ذلك والآخر يختار لحفظ دين الله وشريعته . فأرجو أن يوفقنا لأفضل الاختيار و وإخلاص النية له والعمل الصالح .   وأختار هذه الكلية كي أتعلم ديني الذي هو لحمي ودمي . والعلم معروف شأنه في كتاب الله وسنة رسوله صلى الله عليه وسلم . لأن العلم يهدي الإنسان إلى الخير والتقوى والعمل الصالح . وبهذا العلم أريد رفع الجهل عن نفسي وعن غيري وأكون كالرعيل الأول من الصحابة والتابعين . لأن بالعلم الشرعي والعمل الصالح الناتج عنه كانوا صالحين في أنفسهم ومصلحين لغيرهم . فهم قدوة لنا . ولذلك تعلم الشريعة ليس تخصصا فقط بل هو أكثر من ذلك لأنه متعلق بدين المسلم وسلوكه في هذه الحياة وسفينة نجاته .', ' رحلة إلى أبها قمنا برحلة إلى أبها ، كنا تسعة أشخاص ، كان م

In [8]:
ar_stopwords = set(nltk.corpus.stopwords.words("arabic"))
print(ar_stopwords)

{'أو', 'حاشا', 'لهن', 'خلا', 'هيا', 'مهما', 'لك', 'إليكن', 'لو', 'أنتم', 'هاتي', 'يا', 'فيها', 'آي', 'حتى', 'فيم', 'كذا', 'كأي', 'هؤلاء', 'وإذ', 'أوه', 'أم', 'لهما', 'متى', 'ذلك', 'فلا', 'عليك', 'إنما', 'ليس', 'فمن', 'كي', 'أنتن', 'نعم', 'كل', 'لكيلا', 'كلما', 'مع', 'هاهنا', 'ذلكم', 'لما', 'فإن', 'هن', 'فيما', 'منه', 'تلك', 'في', 'كأنما', 'التي', 'قد', 'كيت', 'دون', 'ممن', 'بهن', 'مما', 'ها', 'هيت', 'ما', 'ذه', 'كلتا', 'هما', 'هاتان', 'أن', 'ذواتي', 'هاتين', 'ولو', 'عليه', 'شتان', 'لها', 'غير', 'ليستا', 'إي', 'إليك', 'هلا', 'هو', 'ذينك', 'ذات', 'أولئك', 'بيد', 'إلا', 'لم', 'بمن', 'ذي', 'نحن', 'أف', 'بين', 'كأن', 'ذا', 'كليهما', 'لعل', 'لي', 'هذا', 'فإذا', 'لستم', 'وإن', 'لاسيما', 'بما', 'هي', 'ألا', 'ذوا', 'به', 'ولا', 'اللذين', 'لسنا', 'هذين', 'هنالك', 'هذه', 'اللتان', 'ليسوا', 'لكي', 'ذلكن', 'منذ', 'حين', 'هيهات', 'أين', 'بخ', 'بماذا', 'أقل', 'كلا', 'سوى', 'هكذا', 'كأين', 'إن', 'إذ', 'كذلك', 'لست', 'بهما', 'ثمة', 'هاك', 'أنت', 'أينما', 'لسن', 'نحو', 'بنا', 'عسى', 'إليكما', 'الذين', '

In [9]:
ar_punc = '،؟-'

In [10]:
def get_all_words(line):
    tokens = word_tokenize(line)
    remove_stop_words = [word for word in tokens if not word in ar_stopwords]
    remove_punc =  [word for word in remove_stop_words if word not in punc]
    remove_ar_pun = [word for word in remove_punc if word not in ar_punc]
    return remove_ar_pun

In [11]:
words = [get_all_words(x) for x in lines]
print(words)

[['التخصص', 'الحمد', 'لله', 'وفقني', 'لدراسة', 'شرعية', 'جامعة', 'الإمام', 'محمد', 'بن', 'سعود', 'الإسلامية', 'فأنا', 'الآن', 'أدرس', 'المستوى', 'الثاني', 'قسم', 'اللغة', 'العربية', 'والعلوم', 'الإسلامية', 'وأريد', 'الالتحاق', 'بكلية', 'الشريعة', 'شاء', 'الله', 'واختيار', 'الناس', 'لهذه', 'الكلية', 'تختلف', 'شخص', 'آخر', 'يختار', 'البعض', 'أجل', 'وظيفة', 'يحصل', 'عليها', 'انتهاء', 'الكلية', 'والبعض', 'الآخر', 'يختار', 'الكلية', 'لأن', 'والديه', 'يريدان', 'والآخر', 'يختار', 'لحفظ', 'دين', 'الله', 'وشريعته', 'فأرجو', 'يوفقنا', 'لأفضل', 'الاختيار', 'و', 'وإخلاص', 'النية', 'والعمل', 'الصالح', 'وأختار', 'الكلية', 'أتعلم', 'ديني', 'لحمي', 'ودمي', 'والعلم', 'معروف', 'شأنه', 'كتاب', 'الله', 'وسنة', 'رسوله', 'صلى', 'الله', 'وسلم', 'لأن', 'العلم', 'يهدي', 'الإنسان', 'الخير', 'والتقوى', 'والعمل', 'الصالح', 'وبهذا', 'العلم', 'أريد', 'رفع', 'الجهل', 'نفسي', 'وعن', 'غيري', 'وأكون', 'كالرعيل', 'الأول', 'الصحابة', 'والتابعين', 'لأن', 'بالعلم', 'الشرعي', 'والعمل', 'الصالح', 'الناتج', 'عنه', 'كانوا', 'ص

In [12]:
new_words = list()
for w in words:
    new_words+=w
print(new_words)

['التخصص', 'الحمد', 'لله', 'وفقني', 'لدراسة', 'شرعية', 'جامعة', 'الإمام', 'محمد', 'بن', 'سعود', 'الإسلامية', 'فأنا', 'الآن', 'أدرس', 'المستوى', 'الثاني', 'قسم', 'اللغة', 'العربية', 'والعلوم', 'الإسلامية', 'وأريد', 'الالتحاق', 'بكلية', 'الشريعة', 'شاء', 'الله', 'واختيار', 'الناس', 'لهذه', 'الكلية', 'تختلف', 'شخص', 'آخر', 'يختار', 'البعض', 'أجل', 'وظيفة', 'يحصل', 'عليها', 'انتهاء', 'الكلية', 'والبعض', 'الآخر', 'يختار', 'الكلية', 'لأن', 'والديه', 'يريدان', 'والآخر', 'يختار', 'لحفظ', 'دين', 'الله', 'وشريعته', 'فأرجو', 'يوفقنا', 'لأفضل', 'الاختيار', 'و', 'وإخلاص', 'النية', 'والعمل', 'الصالح', 'وأختار', 'الكلية', 'أتعلم', 'ديني', 'لحمي', 'ودمي', 'والعلم', 'معروف', 'شأنه', 'كتاب', 'الله', 'وسنة', 'رسوله', 'صلى', 'الله', 'وسلم', 'لأن', 'العلم', 'يهدي', 'الإنسان', 'الخير', 'والتقوى', 'والعمل', 'الصالح', 'وبهذا', 'العلم', 'أريد', 'رفع', 'الجهل', 'نفسي', 'وعن', 'غيري', 'وأكون', 'كالرعيل', 'الأول', 'الصحابة', 'والتابعين', 'لأن', 'بالعلم', 'الشرعي', 'والعمل', 'الصالح', 'الناتج', 'عنه', 'كانوا', 'صا

In [54]:
v = 0
unique_word  =[]
for i in new_words:
    if i not in unique_word:
        v+=1  
        unique_word.append(i)

In [55]:
print(v)

7916


In [13]:
Words_counts = Counter(new_words)

In [14]:
Words_counts

Counter({'التخصص': 115,
         'الحمد': 76,
         'لله': 89,
         'وفقني': 1,
         'لدراسة': 4,
         'شرعية': 2,
         'جامعة': 59,
         'الإمام': 26,
         'محمد': 56,
         'بن': 32,
         'سعود': 35,
         'الإسلامية': 95,
         'فأنا': 2,
         'الآن': 28,
         'أدرس': 41,
         'المستوى': 11,
         'الثاني': 15,
         'قسم': 13,
         'اللغة': 167,
         'العربية': 199,
         'والعلوم': 6,
         'وأريد': 5,
         'الالتحاق': 5,
         'بكلية': 19,
         'الشريعة': 108,
         'شاء': 45,
         'الله': 395,
         'واختيار': 1,
         'الناس': 107,
         'لهذه': 18,
         'الكلية': 34,
         'تختلف': 3,
         'شخص': 5,
         'آخر': 13,
         'يختار': 3,
         'البعض': 1,
         'أجل': 4,
         'وظيفة': 2,
         'يحصل': 5,
         'عليها': 6,
         'انتهاء': 10,
         'والبعض': 1,
         'الآخر': 2,
         'لأن': 103,
         'والديه': 1,
         'يريدان': 1,


In [15]:
Words_counts['التخصص']

115

In [16]:
Words_counts['اللغة العربية']

0

# calculate probability of each word 

In [17]:
def probability(word):
    N = sum(Words_counts.values())
    return Words_counts[word]/N

In [18]:
#check if the word in the corpus
def if_known(words):
    revelant = []
    for w in words:
        if w in Words_counts:
            revelant.append(w)
    return revelant

# Minimun Edit Distance of Only one change

In [20]:
def edits_1(word):
    letters_english = 'abcdefghijklmnopqrstuvwxyz'
    letters_arbic = 'ابتثجحخدذرزسشصضطظعغفقكلمنهويةىء'
    splits = [(word[:i],word[i:]) for i in range(len(word)+1)]
    deletes = [left + right[1:] for left,right in splits if right]
    inserts = [left + center + right for left , right in splits for center in letters_arbic]
    replaces = [left + center + right[1:] for left , right in splits if right for center in letters_arbic]
    transposes = [left + right[1] + right[0] +right[2:] for left , right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

    

# Minimun Edit Distance of Two changes

In [21]:
def edits_2(word):
    for edit1 in edits_1(word):
        for edit2 in edits_1(edit1):
            return edit2

# Returning Candidates for the misspelled word

In [22]:
def candidates(word):
    return ( if_known(edits_1(word)) or if_known(edits_2(word)) or if_known([word]) or [word])


In [23]:
def best_correction(word):
    return max(candidates(word),key = probability)

In [24]:
len(words)

310

In [25]:
max(Words_counts , key=probability)

'الله'

In [26]:
probability('العربية')

0.008852706970950665

In [27]:
if_known(edits_1('اللغ'))

['ابلغ', 'اللغة', 'الله']

In [28]:
candidates('اللغ')

['ابلغ', 'اللغة', 'الله']

In [29]:
best_correction('اللغ')

'الله'

# Making Combinations for N-Gram Model 

In [30]:
def creat_Ngram(n,words):
    words_len = len(words)
    Ngram_list = []
    for idx , word in enumerate(words):
        single_Ngram = []
        for i in range(n):
            if(idx+n <= words_len):
                single_Ngram.append(words[idx+i])
        if(len(single_Ngram)==n):
            Ngram_list.append(tuple(single_Ngram))
    return Ngram_list

# Calculate Count of Uni-Grams and Bi-Grams

In [31]:
def get_counts(N):
    words = new_words
    Ngram = creat_Ngram(N,words)
    Ngram_count = Counter(Ngram)
    return Ngram_count

In [32]:
get_counts(3)

Counter({('التخصص', 'الحمد', 'لله'): 1,
         ('الحمد', 'لله', 'وفقني'): 1,
         ('لله', 'وفقني', 'لدراسة'): 1,
         ('وفقني', 'لدراسة', 'شرعية'): 1,
         ('لدراسة', 'شرعية', 'جامعة'): 1,
         ('شرعية', 'جامعة', 'الإمام'): 1,
         ('جامعة', 'الإمام', 'محمد'): 9,
         ('الإمام', 'محمد', 'بن'): 18,
         ('محمد', 'بن', 'سعود'): 19,
         ('بن', 'سعود', 'الإسلامية'): 15,
         ('سعود', 'الإسلامية', 'فأنا'): 1,
         ('الإسلامية', 'فأنا', 'الآن'): 1,
         ('فأنا', 'الآن', 'أدرس'): 1,
         ('الآن', 'أدرس', 'المستوى'): 1,
         ('أدرس', 'المستوى', 'الثاني'): 1,
         ('المستوى', 'الثاني', 'قسم'): 1,
         ('الثاني', 'قسم', 'اللغة'): 1,
         ('قسم', 'اللغة', 'العربية'): 2,
         ('اللغة', 'العربية', 'والعلوم'): 4,
         ('العربية', 'والعلوم', 'الإسلامية'): 2,
         ('والعلوم', 'الإسلامية', 'وأريد'): 1,
         ('الإسلامية', 'وأريد', 'الالتحاق'): 1,
         ('وأريد', 'الالتحاق', 'بكلية'): 1,
         ('الالتحاق', 'بكلية', '

In [65]:
get_counts(3).values()

dict_values([1, 1, 1, 1, 1, 1, 9, 18, 19, 15, 1, 1, 1, 1, 1, 1, 1, 2, 4, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 57, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 17, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 3, 6, 11, 14, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Calculate the probabilities of all Candidates for a given word

In [33]:
def get_previous(word):
    words = new_words
    previous = dict()
    for w in range(len(words)):
        previous[words[w]] = words[w-1]
    return previous[word]

In [34]:
get_previous('محمد')

'الامام'

In [35]:
def Bigram_prob(word):
    uni_lambda = 0.4
    bi_lambda = 0.6
    candidate_prob_all = dict()
    candidate_words = candidates(word)
    unigram_counts = get_counts(1)
    bigram_counts = get_counts(2)
    prev_word = get_previous(word)
    for w in candidate_words:
        unigram_prob = unigram_counts[(word,)]/len(unigram_counts)
        if bigram_counts[(prev_word,word)]!=0:
            bigram_prob = bigram_counts[(prev_word,word)]/unigram_counts[(prev_word),]
        else:
            bigram_prob = 0
        candidate_prob = (unigram_prob * uni_lambda)+(bigram_prob * bi_lambda)
        if candidate_prob != 0:
            candidate_prob_all[word] = candidate_prob
    return candidate_prob_all

In [36]:
Bigram_prob('الله')

{'الله': 0.6066262422098703}

In [37]:
def correct(word):
   #for vocab in arabic_Correct_words:
    #    if(word == vocab):
     #       print('This word is correct')
      #  else:
    correct_words = candidates(word)
    correct = dict()
    for err in correct_words:
        correct[err] = Bigram_prob(err)
    return correct

# Detect if the word is correct. if not then correct it

In [38]:
correct('اللغ')

{'ابلغ': {'ابلغ': 0.017697589394524864},
 'اللغة': {'اللغة': 0.5124386053562405},
 'الله': {'الله': 0.6066262422098703}}

# Context correction process

In [39]:
import itertools
import math, collections


In [172]:
def candidate_sentence(sentence):

    candidate_sentences = []
    words_count = {}
    for word in sentence.split():
        candidate_sentences.append(candidates(word))
        words_count[word] = candidates(word)
    candidate_sentences = list(itertools.product(*candidate_sentences))
    return candidate_sentences, words_count


In [41]:
candidate_sentence('اللغ العربية')

([('ابلغ', 'العربيه'),
  ('ابلغ', 'العربي'),
  ('ابلغ', 'بالعربية'),
  ('ابلغ', 'اللعربية'),
  ('ابلغ', 'التربية'),
  ('ابلغ', 'الغربية'),
  ('ابلغ', 'والعربية'),
  ('ابلغ', 'العربية'),
  ('اللغة', 'العربيه'),
  ('اللغة', 'العربي'),
  ('اللغة', 'بالعربية'),
  ('اللغة', 'اللعربية'),
  ('اللغة', 'التربية'),
  ('اللغة', 'الغربية'),
  ('اللغة', 'والعربية'),
  ('اللغة', 'العربية'),
  ('الله', 'العربيه'),
  ('الله', 'العربي'),
  ('الله', 'بالعربية'),
  ('الله', 'اللعربية'),
  ('الله', 'التربية'),
  ('الله', 'الغربية'),
  ('الله', 'والعربية'),
  ('الله', 'العربية')],
 {'اللغ': ['ابلغ', 'اللغة', 'الله'],
  'العربية': ['العربيه',
   'العربي',
   'بالعربية',
   'اللعربية',
   'التربية',
   'الغربية',
   'والعربية',
   'العربية']})

In [124]:
def getGram1Prob(word1):
    vocab = sum(Words_counts.values())
    unigramProb = ( Words_counts[word1] + 1 ) / ( vocab + v )
    return unigramProb  

In [43]:
def convertTuple(tup):
    str =  ' '.join(tup)
    return str

In [70]:
def getGram2Prob(word1, word2):
    bigram_count = 0
    vocab = new_words
    bi_sentence = word1+" "+word2
    bigram = creat_Ngram(2,vocab)
    bigramCounter = Counter(bigram) 
    for element in bigramCounter.elements():
        if convertTuple(element).__eq__(bi_sentence):
            #print(convertTuple(element))
            bigram_count +=1
    uniCounter = Words_counts[word1]            
    bigramProb = ( bigram_count + 1 ) / ( uniCounter + v )
    return bigramProb, bigram_count

In [163]:
#test only dont run
#bigram_count = 0
#vocab = new_words
#word1 = 'اللغة'
#word2 = 'العربية'
#bi_sentence = word1+" "+word2
#bigram = creat_Ngram(2,vocab)
#bigramCounter = Counter(bigram)
#for element in bigramCounter.elements():
#    if convertTuple(element).__eq__(bi_sentence):
#        print(convertTuple(element))
#        bigram_count +=1

#uniCounter = Words_counts[word1]            
#bigramProb = bigram_count / uniCounter

In [162]:
#test 
#print(bigram_count)
#print(uniCounter)
#print(bigramProb)
#print(len(bigramCounter))
#print(bigram)


In [125]:
getGram1Prob('العربية')

0.006580029610133245

In [65]:
getGram2Prob('اللغة','العربية')

(0.015959421007051836, 128)

In [194]:
#calculate sentence probability test

print(getGram2Prob('<s>','<اللغة>'))
print(getGram2Prob('اللغة','العربية'))
print(getGram2Prob('العربية','لغة'))
print(getGram2Prob('لغة','عظيمة'))
print(getGram2Prob('عظيمة','</s>'))

(0.00012632642748863063, 0)
(0.015959421007051836, 128)
(0.0004929143561306223, 3)
(0.0001260239445494644, 0)
(0.00012623074981065388, 0)


In [195]:
p1,c1 = getGram2Prob('<s>','<اللغة>')
p2,c2 = getGram2Prob('اللغة','العربية')
p3,c3 = getGram2Prob('العربية','لغة')
p4,c4 = getGram2Prob('لغة','عظيمة')
p5,c5 = getGram2Prob('عظيمة','</s>')

sentence_prob = (p1 * p2 * p3 * p4 * p5)
log_prob = math.log(p1) + math.log(p2) + math.log(p3) + math.log(p4) + math.log(p5)
print(sentence_prob)
print(log_prob)

1.5808877855600217e-17
-38.6859600020658


In [66]:
def getGram3Prob(word1, word2, word3):
    trigram_count = 0
    vocab = new_words
    tri_sentence = word1+" "+word2+" "+word3
    trigram = creat_Ngram(3,vocab)
    trigramCounter = Counter(trigram) 
    for element in trigramCounter.elements():
        if convertTuple(element).__eq__(tri_sentence):
            #print(convertTuple(element))
            trigram_count +=1
    bigramProb, bigramCount = getGram2Prob(word1,word2)           
    trigramProb = (trigram_count + 1) / ( bigramCount + v )
    return trigramProb
    

In [133]:
#test for 3 n gram
#trigram_count = 0
#vocab = new_words
#word1 ='قسم'
#word2= 'اللغة'
#word3='العربية'
#tri_sentence = word1+" "+word2+" "+word3
#trigram = creat_Ngram(3,vocab)
#trigramCounter = Counter(trigram) 
#for element in trigramCounter.elements():
#    if convertTuple(element).__eq__(tri_sentence):
#        #print(convertTuple(element))
#        trigram_count +=1
#bigramProb, bigramCount = getGram2Prob(word1,word2)           
#trigramProb = trigram_count / bigramCount

In [161]:
#print(tri_sentence)
#print(trigram_count)
#print(bigramCount)
#print(trigramProb)



In [67]:
getGram3Prob('قسم','اللغة','العربية')

0.0003788835564536499

In [264]:
def candidates_prob(sentence):
    sentence_list = []
    result = 0
    #for word in sentence.split():
     #    sentence_list.append(word)
    for i in range(0,len(sentence) - 1):
       # print(sentence_list[i])
    #    if i < len(sentence)-2:
    #        p2 = getGram3Prob(sentence[i], sentence[i + 1], sentence[i + 2])
        # if i < len(sentence)-1: 
        p3,c = getGram2Prob(sentence[i], sentence[i + 1])
        p4 = getGram1Prob(sentence[i])
    #    result +=  math.log(p4) + math.log(p3) + math.log(p2)
        result+= math.log(p3) + math.log(p4)
    

    return result

In [265]:
candidates_prob("<s> اللغة العربية لغة عظيمة </s>")

-572.1856544197882

In [158]:
#sentence = " اللغة العربية لغة عظيمة و مليئة لالمعانى الكثيرة" 
#result = 0
#sentence_list = []
#for word in sentence.split():
#    sentence_list.append(word)
#for i in range(0,len(sentence_list) -1):
#    print(sentence_list[i])
#    p3,c = getGram2Prob(sentence_list[i], sentence_list[i + 1])
#    print(p3)
#    result += math.log(p3)
#print(result)


In [159]:
#sentence = " اللغة العربية لغة عظيمة و مليئة لالمعانى الكثيرة" 
#result = 0
#sentence_list = []
#for word in sentence.split():
#    sentence_list.append(word)
#sentence_list.insert(0, '<s>') 
#sentence_list.append('</s>')
#print(sentence_list)

In [207]:
sentence =  "</s> اللغة العربية لغة جميلة <s>" 
result = 0
sentence_list = []
for word in sentence.split():
    sentence_list.append(word)
for i in range(0,len(sentence_list) -1):
    
    print(sentence_list[i])
#    if i < len(sentence_list)-2:
#        p2 = getGram3Prob(sentence_list[i], sentence_list[i + 1], sentence_list[i + 2])
#        print(p2)
#    if i < len(sentence_list)-1:    
    p3,c = getGram2Prob(sentence_list[i], sentence_list[i + 1])
#        print(p3)
    p4 = getGram1Prob(sentence_list[i])
#    print(p4)
#    result +=  math.log(p4) + math.log(p3) + math.log(p2)
    result += math.log(p3) + math.log(p4)
print(result)


   

</s>
اللغة
العربية
لغة
جميلة
-38.687473624958216


In [268]:
def correction_score(old_sentence, new_sentence) :
        # """
        #   Take a old sentence and a new sentence, for each words in the new sentence, if it's same as the orginal sentence, assign 0.95 prob
        #   If it's not same as original sentence, give 0.05 / (count(similarword) - 1)
        # """
    score = 1
    for i in range(len(new_sentence)) :
        if new_sentence[i] in Words_counts:
            score *= 0.95
        else :
            score *= (0.05 / (Words_counts[old_sentence[i]] - 1))
    return math.log(score)


In [233]:
old_sentence = 'اللغ العربية لغة جميلة'
old_sentence_list =[]
for w in old_sentence.split():
    old_sentence_list.append(w)
    
sentences, words_count = candidate_sentence(old_sentence)
score = 1
for new_sentence in sentences:
    new_sentence = list(new_sentence)
    for i in range(len(new_sentence)) :
        if new_sentence[i] in Words_counts:
            score *= 0.95
        else :
            score *= (0.05 / (Words_counts[old_sentence_list[i]] - 1))
result = math.log(score)
print(result)

-98.48312522409712


In [169]:
def return_best_sentence(old_sentence) :
        # """
        #   Generate all candiate sentences and
        #   Calculate the prob of each one and return the one with highest probability
        #   Probability involves two part 1. correct probability and 2. language model prob
        #   correct prob : p(c | w)
        #   language model prob : use stupid backoff algorithm
        # """
    bestScore = 
    bestSentence = []
    old_sentence = [word for word in old_sentence.split()]
    sentences, word_count = candidate_sentence(old_sentence)
    for new_sentence in sentences:
        new_sentence = list(new_sentence)
        score = candidates_prob(new_sentence)
        new_sentence.insert(0, '<s>')
        new_sentence.append('</s>')
        score += score(new_sentence)
        if score >= bestScore:
            bestScore = score
            bestSentence = new_sentence
    bestSentence = ' '.join(bestSentence[1:-1])
    return bestSentence, bestScore
    
    

SyntaxError: invalid syntax (<ipython-input-169-ca615b0711fd>, line 9)

In [271]:
old_sentence = 'اللغ العربية'
bestScore = 0.0
bestSentence = []

sentences, word_count = candidate_sentence(old_sentence)
for new_sentence in sentences:
        new_sentence = list(new_sentence)  
        sentence_score = correction_score(old_sentence, new_sentence) #calculate the word correction
        new_sentence.insert(0, '<s>')
        new_sentence.append('</s>')
        sentence_score += candidates_prob(new_sentence)    #calculate language model probability and word correction p(x|w)
        if sentence_score <= bestScore:
            bestScore = sentence_score
            bestSentence = new_sentence
bestSentence = ' '.join(bestSentence[1:-1])
print(bestSentence)
print(bestScore)

ابلغ والعربية
-56.61256898526901


In [171]:
print(old_sentence)
print(sentences)
print(new_sentence)
print(score)

['جميلة', 'هى', 'اللغ', 'العربية']
[()]
['<s>', '</s>']
0
